In [1]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import math
from scipy import stats
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

# External Impermanent Loss Protection

Withdrawals of TKN during a deficit can result in partial reimbursement in BNT, if the thresholds defined by hlim and hmax are exceeded. This is apparent in the algorithm output T, which represents the number of BNT tokens that should be created for the user. In all of the examples examined above, there is an input variable, w, that has been completely neglected, and which gives the protocol access to TKN of last resort.

The external impermanent loss protection feature of Bancor 3 provides token projects with the means to shoulder part of the insurance burden of the network. This is done by providing a fixed quantity of TKN to an external contract - not the vault - which the protocol can use to subsidize users directly in TKN if, and only if they would otherwise receive a partial reimbursement in BNT. The number of TKN provided to this contract is the w variable.

To explore this behavior thoroughly, at least two examples are required. Since the behavior of the external protection wallet is only relevant in cases where a user has received BNT from the protocol while withdrawing TKN, it seems appropriate to revisit the two examples above where this phenomenon has already played out (i.e. the previous two scenarios where either of the hlim or hmax tests failed. Consider the situation where the TKN trading pool has1,000 BNT (a) and 1,000 TKN (b) in available liquidity, the vault balance of TKN is 1,800 TKN (b+c), the balance of TKN on the staking ledger is 2,200 TKN, and the user confirms a withdrawal for 100 TKN (x). However, this time, let the external protection balance be 1,000 TKN.

![](https://lh6.googleusercontent.com/SJ__QnakcBzeu046IlUVHOAxAODGpjKeIJ7YeHHdW3lRUxGoWf1Wtz5UxlJhc09Bf6vSHbMRU-HRY65K5f9_5Z8oiCy8_SaCFOZtiXf_Ze92-2fzEJ3skUuWtPF-FEFFS3H4mEyh)

The best way to describe the logic of the external protection wallet is to follow the same logic as before, to generate an intermediate set of T and U outputs; recall that T is the number of BNT tokens the user is to receive, and U is the number of TKN the user is receiving from the external protection wallet. The algorithm then determines precisely the quantity by which the T output can be reduced, by substituting BNT for TKN from the external protection contract. First, the protocol checks if the total TKN in the contract (w) is sufficient to replace all of the BNT tokens the user is receiving:

![](https://lh5.googleusercontent.com/O5FhRPNeFoiUxA5GYqz6P2wWasweWgj0BC9K-hrdvfY5hNieLJv3VROI0ozqww4tSatKRqvbxhoH8Rggsfw9qNyPYCoqwmmeu30bzyzbuZV-mR6UZEBWJdh-T1CqXFXBlGR9p9Xe)

In this case the inequality is True - there is more than sufficient TKN available from the external protection wallet to substitute the entire BNT allotment the user would have received. The process is relatively intuitive - the new BNT amount (T†) is set to 0, and its value is converted into TKN units, which are then withdrawn from the external protection wallet.

![](https://lh6.googleusercontent.com/TNTA5GTiVjOTxHt7cuarUaf4OY6nUocWkT58NxuVPeCKjVcKxIw7-xww3e49GlCwdy97Q4IYMsjYJLbjthhgPim003bkNiZN7cBIlxhMfl4IvxTIaIJKEBED-2mu2ifKnIEf4RPD)

![](https://lh5.googleusercontent.com/pF63-XdN0yXjZGm3y66qamVTvdAzk-gm4HtuDeay9kfKHTNi-aimf9lfiTqd796jejzbjMfHkIEa4jSVmtEueFDExPhOFmnrxQchnhVkyuy85mc8fSd_PS5GgMUBo2mpqkI28Awb)

![](https://lh5.googleusercontent.com/OHFH1gvwuSmmlsUFHrPuc-GD0ayuCyhrXoU4kxHAWVbdKW_mP-XCUUAFVfCBabiYXlPlu22MdhMW436AF9XCqBRlf9wE0HJD8rrlMYAJ7W8QEXE-AntJr13fwqRy7R218485Y36p)

![](https://lh4.googleusercontent.com/F7FdNk9VdmGyy6LncpVCjOH3W3jWtjJuTj5KrTpXVLftlthxh6EVyFsjvOqiYBgvwfJ0YJoTNswAKWf17SqbWhALOPc5_2P_GRyyLbl_dwVIoVSVWai89hz3ncnJtF_UuaGoPhPS)

Lastly, let the BNT trading liquidity (a) be 1,800 BNT, let the TKN trading liquidity (b) be 1,800 TKN, let the vault balance of TKN (b+c) be 1,850 TKN, and let the balance of TKN on the staking ledger (e) be 2,200 TKN, and assume the user confirms a withdrawal for 100 TKN (x). However, this time, let the external protection balance be 10 TKN.

![](https://lh4.googleusercontent.com/MnRhs_Xyp95hQuWh99IyYg91-Q6uo--cX__RJLvHglYPGh8DJ9B71n_7F5Xrru8xYV8eaq5iEjeVsFbOr8O-9o77GgoTX4_rYPlGgrMgejHF4UJDlzfWC42A6PRDfxfm2NRtriOq)

The process is similar to the example above; however, there is insufficient TKN in the external protection contract to offset the entirety of the BNT reimbursement received by the user. Therefore, the protocol offsets the maximum allowed by the remaining TKN balance of the external protection contract, and then reimburses the difference with BNT:

![](https://lh3.googleusercontent.com/Nbz9AhuBECMyjVHLxQ8GsWjDvki_FwdXaQS_nXhOSDrV54K6CvKOGE9WRrifIst4GjkwZiCUQzYz8ViJC-DJOdk2TUNorIvyZWMISqvCNVx9gfcQ9ILnTkXOS1zXZ3H2rFBq-E6O)

![](https://lh4.googleusercontent.com/lr43O0EyKgU5BEhw97mMxx28D2Ztj-eWfdBDM8NK-gxCPggrfLWHh9vu2E6NRWB-rTKcyUNBHfWxGMEwpoUmxubSZ6UH9Krt-GkGe2QRNVv6qLtfuGmViMp0fhVotDsXo7HxH3N6)

![](https://lh3.googleusercontent.com/aOz8EP8W495pPj54LVacGcKxovVX3O4zSCMjkMAHiBVQmZfWVfotgdTPx790TgbL918zuMXbKfCyENlh6hufFVrfKBJb-h2gQhAHgWQw63pWqTyWng0On2JpJ3LeQXBsgbyjZaYY)

![](https://lh4.googleusercontent.com/LI31IA972bnGrQ5J3TwKKKKd6JvCzy_Mt3DzYK8mbeSk7ge_39e7qLNyf2IWPQbPUP9DRxO8OnzwGJChiZRTbiKfKPk1FGNRqNrYurrsWDaEoJ4m9XsMdIl_wth18zb3X_ZjdKx5)

In summary, the external protection method provides a higher level of assurance that TKN stakes can be withdrawn entirely in TKN, so long as there is sufficient TKN in the protection contract to support it. The method takes the amount of BNT being minted for the user as input, and the BNT to TKN exchange rate, effectively swapping out an equal quantity of BNT value for TKN value, until there is no TKN remaining.